## Import and Load data file

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
nltk.download('punkt')
nltk.download('wordnet')
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sahme\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sahme\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [9]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatize each word and remove duplicate words from the list

In [10]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

94 documents
12 classes ['allergies', 'exercise', 'good_response', 'goodbye', 'greeting', 'healthy_diet', 'how_are_you', 'nutrients', 'nutritional_supplements', 'thanks', 'vegetarian', 'weight_loss']
87 unique lemmatized words ["'m", "'s", 'a', 'about', 'activity', 'allergen-free', 'allergy', 'am', 'anyone', 'are', 'balanced', 'bye', 'constitutes', 'contribute', 'day', 'dealing', 'diet', 'dietary', 'doe', 'doing', 'eat', 'effective', 'enough', 'essential', 'exercise', 'food', 'for', 'get', 'good', 'goodbye', 'guide', 'health', 'healthy', 'hello', 'helpful', 'hey', 'hi', 'how', 'i', 'im', 'importance', 'important', 'in', 'is', 'later', 'lifestyle', 'lose', 'loss', 'lot', 'macronutrients', 'me', 'micronutrient', 'mineral', 'necessary', 'nutrient', 'nutrition', 'nutritional', 'nutritious', 'of', 'okay', 'physical', 'protein', 'recommendation', 'see', 'should', 'source', 'strategy', 'sufferer', 'supplement', 'take', 'tell', 'thank', 'thanks', 'that', 'there', 'thnks', 'thx', 'tip', 'to', '

## Create training and testing data

In [16]:
training = []
output_empty = [0] * len(classes)

In [17]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

In [19]:
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1
training.append([bag, output_row])

In [21]:
import numpy as np

# Shuffle our features
random.shuffle(training)

# Extract features (X) and labels (Y) from training data
train_x = np.array([doc[0] for doc in training])
train_y = np.array([doc[1] for doc in training])

# Print to verify the shapes
print("Shape of train_x:", train_x.shape)
print("Shape of train_y:", train_y.shape)


Shape of train_x: (2, 87)
Shape of train_y: (2, 12)


## Create Model

In [22]:
# Importing necessary libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD  # Importing SGD from keras.optimizers
from tensorflow.keras.optimizers.legacy import SGD as legacy_SGD # Importing the legacy SGD from tensorflow.keras.optimizers.legacy

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons, and 3rd output layer contains a number of neurons
# equal to the number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = legacy_SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True) # Use the legacy SGD with the decay argument
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created")



Epoch 1/200


1/1 [==============================] - 4s 4s/step - loss: 2.3757 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 2.2036 - accuracy: 0.5000
Epoch 3/200
1/1 [==============================] - 0s 25ms/step - loss: 2.1177 - accuracy: 0.5000
Epoch 4/200
1/1 [==============================] - 0s 31ms/step - loss: 2.0891 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 16ms/step - loss: 2.2337 - accuracy: 0.5000
Epoch 6/200
1/1 [==============================] - 0s 18ms/step - loss: 1.8853 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 39ms/step - loss: 1.7745 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 17ms/step - loss: 1.1767 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 20ms/step - loss: 1.0055 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 16ms/step - loss: 1.2073 - accuracy: 1.0000
Epoch 11/200
1

c:\Users\sahme\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model created


In [23]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [25]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [29]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [30]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [31]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [32]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [33]:
def send():
    try:
        msg = input("You: ")
        if msg.lower() == 'exit':
            raise KeyboardInterrupt("User exited the chat")
        
        if msg != '':
            res = chatbot_response(msg)
            print("Bot:", res)
    
    except KeyboardInterrupt as e:
        print(e)
        exit()

while True:
    send()


1/1 [==============================] - 0s 380ms/step
Bot: Managing food allergies involves avoiding trigger foods, reading labels carefully, and consulting with a healthcare professional for personalized advice.
1/1 [==============================] - 0s 69ms/step
Bot: Managing food allergies involves avoiding trigger foods, reading labels carefully, and consulting with a healthcare professional for personalized advice.
1/1 [==============================] - 0s 58ms/step
Bot: Managing food allergies involves avoiding trigger foods, reading labels carefully, and consulting with a healthcare professional for personalized advice.
1/1 [==============================] - 0s 317ms/step
Bot: Managing food allergies involves avoiding trigger foods, reading labels carefully, and consulting with a healthcare professional for personalized advice.
1/1 [==============================] - 0s 84ms/step
Bot: Managing food allergies involves avoiding trigger foods, reading labels carefully, and consulting